<a href="https://colab.research.google.com/github/mattheweisenberg6/MAT421/blob/main/ModuleC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**19.1 Root Finding Problem Statement**

For some functions f(x), it can be difficult to calculate its root, thus we can generate approximations of its root.

In [2]:
#approximating root of f(x) = cos(x) - x near -2

import numpy as np
from scipy import optimize

f = lambda x: np.cos(x) - x
r = optimize.fsolve(f, -2)
print("r =", r)

# Verify the solution is a root
result = f(r)
print("result =", result)


r = [0.73908513]
result = [0.]


In [3]:
#approximating root of f(x) = 1/x which has no root

f = lambda x: 1/x

r, infodict, ier, mesg = optimize.fsolve(f, -2, full_output=True)
print("r =", r)

result = f(r)
print("result=", result)

print(mesg)

r = [-3.52047359e+83]
result= [-2.84052692e-84]
The number of calls to function has reached maxfev = 400.


The value r we got is not the root even though f(r) is a small number. Since full_output is on, we have more information about the output which tells us the cause for failure: "The number of calls to function has reached maxfev = 400."

**19.2 Tolerance**

 Tolerance is the level of error that is acceptable for an engineering application. When a computer converges on a solution, it has chosen a solution that is less than the tolerance. For computing roots numerically, it's important to have metrics for errors and tolerance. For computing roots, we want x_r to be such that f(x_r) = 0 which means |f(x)| is possible metric for error.

Let e = |f(x)| and tol be the acceptable level of error. The function f(x) = x^ 2 + tol/2 has no real roots. However, f(0) = tol/2 and is therefore an acceptable solution for the root finding program.

Let e = |x_i+1 - x_i| and tol be the acceptable level of error. Then function 1/x has no real roots, but the guesses x_i = -tol/4 and x_i+1 = tol/4 have an error of e = tol/2 and therefore are acceptable solutions.

**19.3 Bisection**

Intermediate Value Theorem states that if f(x) is a continuous function between a and b and sign(f(a) != f(b)), then there must be a c, such that a < c < b and f(c) = 0. Bisection method uses the intermediate value theorem iteratively to find roots. If there is some f(a) > 0 and f(b) < 0, then by intermediate value theorem there is some root on the open interval (a,b). Let m = (a+b)/2. If f(m) = 0 or is close enough, then m is a root. If f(m) > 0, then there is a root in the open interval (m,b), else if f(m) < 0 then there is one on the open interval (a,m). We repeat process until m is acceptably low.

In [5]:
#function that approximates root r of f, bounded by a and b within  |f((a+b)/2)| < tol.

import numpy as np

def my_bisection(f, a, b, tol):
    # approximates a root, R, of f bounded
    # by a and b to within tolerance
    # | f(m) | < tol with m the midpoint
    # between a and b Recursive implementation

    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")

    # get midpoint
    m = (a + b)/2

    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on a.
        # Make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on b.
        # Make recursive call with b = m
        return my_bisection(f, a, m, tol)

In [6]:
#sqrt(2) can be computed as root of function f(x) = x^2-2 starting a = 0 and b = 2.
#We can use bisection to approximate sqrt(2) to a tolerance of |f(x)| < 0.1 and
#|f(x)| < 0.01.

f = lambda x: x**2 - 2

r1 = my_bisection(f, 0, 2, 0.1)
print("r1 =", r1)
r01 = my_bisection(f, 0, 2, 0.01)
print("r01 =", r01)

print("f(r1) =", f(r1))
print("f(r01) =", f(r01))


r1 = 1.4375
r01 = 1.4140625
f(r1) = 0.06640625
f(r01) = -0.00042724609375


In [7]:
#using a = 2 and b = 4

my_bisection(f, 2, 4, 0.01)

Exception: The scalars a and b do not bound a root

**19.4 Newton-Raphson Method**

The method finds roots through iterating Newton steps from x_0 until error < tolerance. This means that each iteration is a guess or approximation, which creates a linear approximation where each guess is improved.

In [8]:
#sqrt(2) is root of function f(x) = x^2 - 2. Use 1.4 at starting point x_0 we can estimate
#sqrt(2)

import numpy as np

f = lambda x: x**2 - 2
f_prime = lambda x: 2*x
newton_raphson = 1.4 - (f(1.4))/(f_prime(1.4))

print("newton_raphson =", newton_raphson)
print("sqrt(2) =", np.sqrt(2))

newton_raphson = 1.4142857142857144
sqrt(2) = 1.4142135623730951


In [10]:
#newton raphson function to estimate roots of f with |f(x)| as error measurement

def my_newton(f, df, x0, tol):
    # output is an estimation of the root of f
    # using the Newton Raphson method
    # recursive implementation
    if abs(f(x0)) < tol:
        return x0
    else:
        return my_newton(f, df, x0 - f(x0)/df(x0), tol)

#estimating sqrt(2) with tolernace of 1e-6 starting at x_0 = 1.5

estimate = my_newton(f, f_prime, 1.5, 1e-6)
print("estimate =", estimate)
print("sqrt(2) =", np.sqrt(2))


estimate = 1.4142135623746899
sqrt(2) = 1.4142135623730951


Since values are close and within tolerance, the solution is converge upon.

In [11]:
#single newton step to get improved approximation of function f(x) = x^3 + 3x^2 -2x -5
#with x_0 = 0.29

x0 = 0.29
x1 = x0-(x0**3+3*x0**2-2*x0-5)/(3*x0**2+6*x0-2)
print("x1 =", x1)

x1 = -688.4516883116648


**19.5 Root Finding in Python**

We can use scipy libraries instead of manual computation which saves time.


In [13]:
from scipy.optimize import fsolve

f = lambda x: x**3-100*x**2-x+100

fsolve(f, [2, 80])

#valid solution for roots

array([  1., 100.])

In [14]:

f = lambda x: x**2-230*x**3-x+1000

fsolve(f, [2, 80])

array([1.63270174, 1.63270174])